# <center>Data and computation 2</center>
### <center>Alfred Galichon (NYU)</center>
## <center>Week 1: logistic regression and generalized linear models</center>


## References

* Savage, L. (1951). The theory of statistical decision. JASA.
* Bonnet, Fougère, Galichon, Poulhès (2021). Minimax estimation of hedonic models. Preprint.

## Loading the libraries

First, let's load the libraries we shall need.

In [2]:
#pip install gurobipy
#!pip install tabulate

You should consider upgrading via the '/Users/fabrizio/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import scipy.sparse as spr
from scipy import optimize, special
import gurobipy as grb
from sklearn import linear_model
from tabulate import tabulate


## Our data
We will go back to the dataset of Greene and Hensher (1997). As a reminder, 210 individuals are surveyed about their choice of travel mode between Sydney, Canberra and Melbourne, and the various costs (time and money) associated with each alternative. Therefore there are 840 = 4 x 210 observations, which we can stack into `travelmodedataset` a 3 dimensional array whose dimensions are mode,individual,dummy for choice+covariates.

Let's load the dataset and take a first glance at it:

In [4]:
thepath = 'https://raw.githubusercontent.com/math-econ-code/mec_optim_2021-01/master/data_mec_optim/demand_travelmode/'
travelmode =  pd.read_csv(thepath+'travelmodedata.csv')
travelmode.head()

,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,1,air,no,69,59,100,70,35,1
1,1,train,no,34,31,372,71,35,1
2,1,bus,no,35,25,417,70,35,1
3,1,car,yes,0,10,180,30,35,1
4,2,air,no,64,58,68,68,30,2


# GLM to estimate discrete choice models

## Estimation with observed heterogeneity

We assume that we observe individual characteristics that are relevant for individual choices, that is $U_{iy}=\sum_k \Phi_{iyk} \beta_k$, or in matrix form<br>
$U = \Phi \beta,$<br>
where $\beta\in\mathbb{R}^{p}$ is a parameter, and $\Phi$ is a $\left(\left\vert \mathcal{I}\left\vert\right\vert\mathcal{Y}\right\vert \right) \times p$ matrix.

Assume $u_{iy}=U_{iy} + \varepsilon _{iy}= \sum_{k}\Phi _{iyk} \beta _{k}+\varepsilon _{iy}$.

Let $\hat{\mu}_{iy}$ be the indicator that $i$ chooses alternative $y$. 



We create a `discreteChoicePb` class where we store that data. An arc $a$ is a pair $iy$.

In [5]:
class DiscreteChoicePb():
    def __init__(self,Φ_i_y_k, μhat_i_y):
        self.nbi,self.nby,self.nbk = Φ_i_y_k.shape
        self.nba = self.nbi * self.nby
        self.Φ_a_k = Φ_i_y_k.reshape((self.nba,-1))
        self.μhat_a = μhat_i_y.flatten()

We build an object `travelEx` based on our travel data and a parametric model where the regressors are `travel`, `-travel*income` and `gcost`.

In [6]:
def prepare_travel_data():
    μhat_a = np.where(travelmode['choice'] =='yes' , 1, 0)
    #nobs,ncols = travelmode.shape
    nby = travelmode['mode'].nunique()
    nbi = travelmode.shape[0] // nby
    covariates = travelmode[['travel', 'income', 'gcost']].values
    Φ_a_k = np.column_stack([ covariates[:,0] , - (covariates[:,0] * covariates[:,1] ), - covariates[:,2] ])
    _,nbk = Φ_a_k.shape 
    Φbar_k = Φ_a_k.mean(axis = 0)
    Φstdev_k = Φ_a_k.std(axis = 0, ddof = 1)
    Φ_i_y_k = ((Φ_a_k - Φbar_k[None,:]) / Φstdev_k[None,:]).reshape((nbi,nby,nbk)) # this is the gradient
    return DiscreteChoicePb(Φ_i_y_k,μhat_a)

travelEx = prepare_travel_data()

## Maximum likelihood estimation

The probability $\mu_{iy}$ that individual $i$ chooses alternative $y$ is given by $\partial G_i / \partial U_y (\Phi \beta)$. 


The log-likelihood function is given by

$
l\left(  \beta\right)  =\sum_{y}\hat{\mu}_{iy}\log \mu_{iy}\left(\Phi \beta\right) = \sum_{y}\hat{\mu}_{iy}\log  
\frac {\partial G_i}  {\partial U_y} (\Phi \beta)
$

A common estimation method of $\beta$ is by maximum likelihood: $\max_{\beta}l\left(  \beta\right) $; MLE is statistically efficient; the problem is that the problem is not guaranteed to be convex, so there may be computational difficulties (e.g. local optima).



### MLE, logit case

In the logit case, the log-likelihood associated with observation $i\in\mathcal{I}$ is

$
l_{i}\left( \beta \right) =\sum_{y\in \mathcal{Y}}\hat{\mu}_{iy}\left( \Phi
\beta \right) _{iy}-\log \sum_{y\in \mathcal{Y}}\exp \left( \Phi \beta
\right) _{iy}$

and the max-likelihood rewrites as

$\max_{\beta }\left\{ \sum_{i\in \mathcal{I},y\in \mathcal{Y}}\hat{\mu}%
_{iy}\left( \Phi \beta \right) _{iy}-\sum_{i\in \mathcal{I}}\log \sum_{y\in 
\mathcal{Y}}\exp \left( \Phi \beta \right) _{iy}\right\}$

so that the max-likehood boils down to

\begin{align*}
\max_{\beta}\left\{  \hat{\mu}^{\intercal} \Phi \beta- \sum_i G_i\left( \Phi \beta\right)\right\}
\end{align*}

whose value is the Legendre-Fenchel transform of $\beta\rightarrow \sum_i G_i\left( \Phi \beta\right)$ evaluated at $\Phi ^{^{\intercal}}\hat{\mu}$.

Note that the vector $\Phi^{^{\intercal}}\hat{\mu}$ is the vector of empirical moments, which is a sufficient statistics in the logit model.

As a result, in the logit case, the MLE is a convex optimization problem, and it is therefore both statistically efficient and computationally efficient.



### Moment estimation

The previous remark will inspire an alternative procedure based on the moments statistics $\Phi^{^{\intercal}}\hat{\mu}$.

The social welfare is given in general by $W\left(  \beta\right) =\sum_i G_i\left(  \Phi\beta\right)  $. One has <br>$\partial_{\beta^{k}}W\left(\beta\right)  =\sum_{iy} \frac {\partial G_i} {\partial U_y}(\Phi\beta) \Phi_{yik}$, that is 

\begin{align*}
\nabla W\left(  \beta\right)  = \Phi^{\intercal}\nabla G_i\left(  \Phi\beta\right)  ,
\end{align*}

which is the vector of predicted moments.

Therefore the program

$\max_{\beta }\left\{ \hat{\mu}^{\top }\Phi \beta -\sum_{i\in \mathcal{I}%
}G\left( \left( \Phi \beta \right) _{i.}\right) \right\} ,$

(where G is the Emax operator associated with the distribution of the random utility), picks up the parameter $\beta$ which matches the empirical moments $\Phi^{^{\intercal}}\hat{q}$ with the predicted ones $\nabla W\left(\beta\right)  $. This procedure is not statistically efficient, but is computationally efficient becauses it arises as a convex optimization problem.

### Creating a DiscreteChoicePb class

In [7]:
def DiscreteChoicePb_mle_diy(self):
    def minus_log_likelihood(β_k, σ = 1):
        Φβ_i_y = (self.Φ_a_k.dot(β_k)).reshape((-1,self.nby)) 
        maxΦβ_i = Φβ_i_y.max(axis = 1)
        d_i = np.sum(np.exp((Φβ_i_y -maxΦβ_i[:,None])/σ ), axis = 1)
        return - ((Φβ_i_y.flatten()*self.μhat_a).sum() / σ  -  (maxΦβ_i / σ + np.log(d_i)).sum())

    def grad_minus_log_likelihood(β_k, σ = 1):
        Φβ_i_y = (self.Φ_a_k.dot(β_k)).reshape((-1,self.nby)) 
        maxΦβ_i = Φβ_i_y.max(axis = 1)
        d_i = np.sum(np.exp((Φβ_i_y - maxΦβ_i[:,None] )/σ ), axis = 1)
        μβ_iy = (np.exp((Φβ_i_y - maxΦβ_i[:,None] )/σ ) / d_i[:,None]).flatten()
        return  - ((self.μhat_a - μβ_iy).reshape((1,-1)) @ self.Φ_a_k).flatten()

    βtilde0_k = np.zeros(self.nbk)
    res = optimize.minimize(minus_log_likelihood,method = 'CG',jac = grad_minus_log_likelihood, x0 = βtilde0_k )
    βtilde_k = res['x']
    print(-minus_log_likelihood (βtilde_k))
    return βtilde_k[:-1] / βtilde_k[-1],  1 / βtilde_k[-1], - res['fun']  # this is the Jacobian

DiscreteChoicePb.mle_diy = DiscreteChoicePb_mle_diy


Compute using:

In [8]:
βmle_k,Tmle,llmle = travelEx.mle_diy()
print('DIY approach. βmle_k = ',βmle_k,'; Tmle = ',Tmle, ' ; ll=',llmle,'.')

-277.7052141445853
DIY approach. βmle_k =  [0.33826733 0.85179311] ; Tmle =  1.8162760082307419  ; ll= -277.7052141445853 .


### Computation as a Poisson regression using GLM in scikit-learn

As a reminder, this can be computed as 
\begin{align*}
\min_{\beta,u} \left\{ \sum_{iy} \hat{\mu}_{iy} \left(   \Phi\beta - (I_\mathcal{I} \otimes 1_\mathcal{Y}) u \right)  _{iy} - \sum_{iy} \exp\left(  \left(  \Phi\beta - (I_\mathcal{I} \otimes 1_\mathcal{Y}) u \right) _{iy} \right)  \right\}
\end{align*}

which leads to the following call to `scikit-learn`:


In [9]:
def DiscreteChoicePb_mle_glm(self, max_iter = 1000, tol=0.0001):
    poisson = linear_model.PoissonRegressor(alpha = 0, fit_intercept=False,max_iter= max_iter, tol = tol)
    X_a_l = spr.hstack([self.Φ_a_k, -spr.kron(spr.identity(self.nbi), np.ones((self.nby,1)))])
    poisson.fit(X_a_l, self.μhat_a)
    val =  self.μhat_a @ X_a_l @ poisson.coef_ - (np.exp(X_a_l @ poisson.coef_)).sum() + self.nbi
    return poisson.coef_[:self.nbk-1] / poisson.coef_[self.nbk-1],1 / poisson.coef_[self.nbk-1], val

DiscreteChoicePb.mle_glm = DiscreteChoicePb_mle_glm

We verify that both approaches are indeed equivalent:

In [10]:
βmleglm_k,Tmleglm,llmleglm = travelEx.mle_glm(max_iter = 10000, tol = 1e-9)
print('GLM approach. βmle_k = ',βmleglm_k,'; Tmle = ',Tmleglm, ' ; ll=',llmleglm,'.')
print('DIY approach. βmle_k = ',βmle_k,'; Tmle = ',Tmle, ' ; ll=',llmle,'.')

GLM approach. βmle_k =  [0.33826961 0.8517962 ] ; Tmle =  1.816271931055784  ; ll= -277.7052141450895 .
DIY approach. βmle_k =  [0.33826733 0.85179311] ; Tmle =  1.8162760082307419  ; ll= -277.7052141445853 .
